# Radar Classification

## A Radar Classification Basics

In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp
from cvxpy.atoms.affine.wraps import psd_wrap
from read_data import *
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%       MGT - 418         %%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%      Convex Optimization - Project 2          %%%%%%%%%%%%%%
#%%%%%%%%%%%%%%             2021-2022 Fall                    %%%%%%%%%%%%%%
#%%%%%%%%%%%%%%      Learning the Kernel Function             %%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

### (a) Read & Split data

**(5 points)** Read the data file ionosphere.data into memory by using the scriptsreaddata.pyorreaddata.m.Use the code skeletonsmain.ipnyb or main.m to randomly select 80% of the data for training.

In [2]:
data, labels = prepare_ionosphere_dataset()

In [3]:
from process_data import train_test_split

In [5]:
for i in range(3):
    print("#{}".format(i))
    # data_train, data_test, labels_train, labels_test
    data_train, _, _, _ = train_test_split(data, labels, train_size=0.8, random_seed=i)
    print(data_train[:1,:])
    print(np.shape(data_train))

#0
[[1.0 0.97588 -0.10602 0.94601 -0.208 0.92806 -0.2835 0.85996 -0.27342
  0.79766 -0.47929 0.78225 -0.50764 0.74628 -0.61436 0.57945 -0.68086
  0.37852 -0.73641 0.36324 -0.76562 0.31898 -0.79753 0.22792 -0.81634
  0.13659 -0.8251 0.04606 -0.82395 -0.04262 -0.81318 -0.13832 -0.80975]]
(281, 33)
#1
[[1.0 -0.205 0.2875 0.23 0.1 0.2825 0.3175 0.3225 0.35 0.36285 -0.34617
  0.0925 0.275 -0.095 0.21 -0.0875 0.235 -0.34187 0.31408 -0.48 -0.08
  0.29908 0.33176 -0.58 -0.24 0.3219 -0.28475 -0.47 0.185 -0.27104
  -0.31228 0.40445 0.0305]]
(281, 33)
#2
[[1.0 1.0 0.5782 1.0 -1.0 1.0 -1.0 1.0 -1.0 1.0 -1.0 1.0 -1.0 1.0 -1.0
  1.0 -1.0 1.0 -1.0 1.0 -0.62796 1.0 -1.0 1.0 -1.0 1.0 -1.0 1.0 -1.0 1.0
  -1.0 1.0 -1.0]]
(281, 33)


### (b) Define kernel function & Solve QCQP

**(15 points)** Define polynomial, Gaussian, and linear kernel function, and construct the kernel matrices $\hat{K}^l,\ l = 1,2,3$, for all training samples
Solve the QCQP in (5) for $\rho = 2$, $p = 2$, $\sigma = 2$ and $c = \sum_{l=1}^3 \mathrm{tr}(\hat{K}^l) $ with CVXPY and MOSEK in Python or with YALMIP and GUROBI in MATLAB, and record the optimal dualvariables $\mu_1^*$, $\mu_2^*$, and $\mu_3^*$. Use the code skeletons `kernel_learning` (in `main.ipynb`) or `kernel_learning.m`

### (c) Apply kernel trick for SVM prediction

**(10 points)** Use the code skeletons `SVM_predict`(in `main.ipynb`) or `SVM_predict.m`.

> The size of the dual QP is independent of the feature
dimension D!

## B Repeat experiments

**(5 points)** Repeat the steps 4(a)–(c) 100 times with different seeds for the random partition of the data intotraining and test sets, and report the average test accuracy (correct classification rate) to Table 1

| Kernel function  | $\hat{k}^1$ | $\hat{k}^2$ | $\hat{k}^3$ | $\sum_{l=1}^3 \hat{k}^l$ |
| ---------------- | ----------- | ----------- | ----------- | ------------------------ |
| Average accuracy |             |             |             |                          |


## C Solve dual problem

**(10  points)** For  each  of  the  100  training  and  test  sets  constructed  in  5.,  solve  (2)  using  the  kernels  functions $\hat{k}^1$, $\hat{k}^2$, and $\hat{k}^3$, respectively, and report the average test accuracies in Table 1. Use the code skeletons `SVM_predict`(in `main.ipynb`) or `SVM_predict.m`.

| Kernel function  | $\hat{k}^1$ | $\hat{k}^2$ | $\hat{k}^3$ | $\sum_{l=1}^3 \hat{k}^l$ |
| ---------------- | ----------- | ----------- | ----------- | ------------------------ |
| Average accuracy |             |             |             |                          |


---

In [2]:
import pandas as pd
import numpy as np
import cvxpy as cp
from cvxpy.atoms.affine.wraps import psd_wrap
from read_data import *
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%       MGT - 418         %%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%      Convex Optimization - Project 2          %%%%%%%%%%%%%%
#%%%%%%%%%%%%%%             2021-2022 Fall                    %%%%%%%%%%%%%%
#%%%%%%%%%%%%%%      Learning the Kernel Function             %%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

In [75]:
def svm_fit(kernel, y_tr, rho):
    """
    Dual of soft-margin SVM problem (2)
    Use cvxpy.atoms.affine.psd_wrap for each G(\hat K^l) matrix when it appear in the constraints and in the objective
    """
    n_tr = len(y_tr)
    G =  ...
    lambda_ = cp.Variable(n_tr)
    dual_obj = cp.Maximize(... cp.quad_form(lambda_, psd_wrap(G)))
    cons = []
    ...
    prob = cp.Problem(dual_obj, cons)
    prob.solve(solver=cp.MOSEK)
    lambda_opt = lambda_.value
    b_opt =  ...
    return lambda_opt, b_opt


def svm_predict(kernel, y_tr, y_te, lambda_opt, b_opt, rho):
    """
    Predict function for kernel SVM. 
    See lecture slide 183.
    """
    n_te = len(y_te)
    n_tr = len(y_tr)
    ...
    acc = ...
    return acc

def kernel_learning(K1, K2, K3, y_tr, rho):
    """
    Kernel learning for soft margin SVM. 
    Implementation of problem (5)
    Use cvxpy.atoms.affine.psd_wrap for each G(\hat K^l) matrix when it appear in the constraints and in the objective
    """
    ...
    r1 = np.trace(K1) 
    ...
    lambda_ = cp.Variable(n_tr)
    z = cp.Variable(1)
    ...
    
    cons = []
    cons.append(z * r1 >= 1/ (2 * rho) * cp.quad_form(lambda_, psd_wrap(G1)))
    ...
    ...
    prob = cp.Problem(obj, cons)
    prob.solve(solver=cp.MOSEK)

    mu_opt1 = cons[0].dual_value
    ...
    b_opt = ....dual_value
    return mu_opt1, mu_opt2, mu_opt3, lambda_.value, b_opt

In [75]:
acc_opt_kernel = []    
acc_poly_kernel = []    
acc_gauss_kernel = []    
acc_linear_kernel = []    
rho = 0.01
# data, labels = prepare_ionosphere_dataset()
for iters in range(100): 
    ## Please do not change the random seed.
    np.random.seed(iters)
    ### Training-test split
    msk = np.random.rand(data_normalized.shape[0]) <=...
    x_tr = data[...]
    x_te = data[...]
    y_tr = labels[...]
    y_te = labels[...]
 
    n_tr = y_tr.shape[0]
    n_te = y_te.shape[0]
    n_tr = x_tr.shape[0]
    n_te = x_te.shape[0]
    
    x_all = np.vstack([x_tr, x_te])
    n_all = x_all.shape[0]

    ## Prepare the initial choice of kernels 
    # It is recommended to prepare the kernels for all the training and the test data
    # Then, the kernel size will be (n_tr + n_te)x(n_tr + n_te).
    # Use only the training block (like K1[0:n_tr, 0:n_tr] ) to learn the classifier 
    # (for the functions svm_fit and kernel_learning).
    # When predicting you may use the whole kernel as it is. 
    K1 = ...
    K2 = ...
    K3 = ...

    mu_opt1, mu_opt2, mu_opt3, lambda_opt, b_opt = kernel_learning(...)
    opt_kernel = ...
    acc_opt_kernel.append(svm_predict(...))
    
    lambda_opt, b_opt = svm_fit(...)
    acc_poly_kernel.append(svm_predict(...))
    
    lambda_opt, b_opt = svm_fit(...)
    acc_gauss_kernel.append(svm_predict(...))
    
    lambda_opt, b_opt = svm_fit(...)
    acc_linear_kernel.append(svm_predict(...)
    print('Iteration-->' + str(iters))
print('Average dual accuracy with optimal kernel is ' + str(np.mean(acc_opt_kernel)))
print('Average dual accuracy with polynomial kernel is ' + str(np.mean(acc_poly_kernel)))
print('Average dual accuracy with gaussian kernel is ' + str(np.mean(acc_gauss_kernel)))
print('Average dual accuracy with linear kernel is ' + str(np.mean(acc_linear_kernel)))